In [1]:
import duckdb
from os import remove

SF = 1

duckdb.__version__

'v0.7.1-dev648'

In [2]:
database_file = "minimal_snb.duckdb"

con = duckdb.connect(database_file)
schema = open(f"SNB-SF{SF}-dataset/minimal_schema.sql").read().split(";")
for query in schema:
    con.execute(query)
load = open(f"SNB-SF{SF}-dataset/minimal_load.sql").read().split(";")
for query in load:
    con.execute(query)
print(f"Loaded dataset of scale factor {SF}")

con.close()

Loaded dataset of scale factor 1


In [3]:
%load_ext sql


In [4]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///minimal_snb.duckdb

%sql load 'sqlpgq'

""


In [5]:
%%sql
    CREATE PROPERTY GRAPH social_network
    VERTEX TABLES (
        Person LABEL Person
    )
    EDGE TABLES (
        Person_knows_Person 
            SOURCE KEY (Person1id) REFERENCES Person (id)
            DESTINATION KEY (Person2id) REFERENCES Person (id)
            LABEL Person_knows_Person
    )

""


In [9]:
%%sql
    SELECT pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->{1,4}(p2:person)
    WHERE p.firstname = 'Bob'
    COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
    ) pg
    GROUP BY ALL

(duckdb.ParserException) Parser Error: syntax error at or near "\"
LINE 3: ...pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->\(1, 4)(p2:person)
                                                  ^
[SQL: SELECT pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->\(1, 4)(p2:person)
    WHERE p.firstname = 'Bob'
    COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
    ) pg
    GROUP BY ALL]
(Background on this error at: https://sqlalche.me/e/14/f405)


INFO:backoff:Backing off send_request(...) for 0.8s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))


In [7]:
%%sql
            SELECT pg.firstname, pg.id, count(pg.reachable_people)
            FROM GRAPH_TABLE (social_network,
            MATCH (p:Person)-[k:Person_knows_Person]-> *(p2:person)
            WHERE p.firstname = 'Bob'
            COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
            ) pg
            GROUP BY ALL

,firstname,id,count(pg.reachable_people)
0,Bob,977,8385
1,Bob,10995116285001,5232
2,Bob,28587302324999,128
3,Bob,24189255821234,1
